In [2]:

import os
os.environ['P8_PG_HOST'] = 'eepis.percolationlabs.ai'
os.environ['P8_PG_PORT'] = '5434'
os.environ['P8_PG_PASSWORD'] = os.environ.get('P8_TEST_BEARER_TOKEN')
os.environ['P8_USE_AWS_S3'] = 'true'
import percolate as p8
from percolate.models import Resources
from percolate.services import PostgresService
pg = PostgresService()
a = p8.Agent(Resources)
# import nest_asyncio
# nest_asyncio.apply()
#print(os.environ.get('P8_TEST_BEARER_TOKEN'))
import io
from percolate.services import FileSystemService
fs = FileSystemService()
 
list(fs.read_chunks('s3://res-data-platform/gdrive-sync/v1/1dBLUf0DfR7QmVRnGTyrtzg0o67AfUqeU/MakeOne/Market_Size_for_Tech_Packs_and_Industry_Design/Production_Support'))

2025-06-25 11:15:54.609 | DEBUG    | percolate.services.FunctionManager:add_function:42 - added function help
2025-06-25 11:15:54.750 | DEBUG    | percolate.services.FunctionManager:add_function:42 - added function chunked_resource
2025-06-25 11:15:54.756 | DEBUG    | percolate.services.FunctionManager:add_function:42 - added function chunked_resource_from_text
2025-06-25 11:15:54.763 | DEBUG    | percolate.services.FunctionManager:add_function:42 - added function get_recent_uploads_by_user
2025-06-25 11:15:54.768 | DEBUG    | percolate.services.FunctionManager:add_function:42 - added function save_user_fact
2025-06-25 11:15:54.771 | DEBUG    | percolate.services.FunctionManager:add_function:42 - added function search_facts_by_users
2025-06-25 11:15:54.775 | DEBUG    | percolate.services.FunctionManager:add_function:42 - added function get_entities
2025-06-25 11:15:54.780 | DEBUG    | percolate.services.FunctionManager:add_function:42 - added function search
2025-06-25 11:15:54.784 | D

[Resources(id='7094c46c-32f8-54c0-8189-85c80a08282c', name='Production_Support_chunk_1', category='unknown_chunk', content="{'text_pages': ['Market Size for Tech Packs and Industry Design/Production Support  \\nhow much does the global fashion industry spend on people and tools creating and managing tech packs in the production process ChatGPT said: Quantifying the exact global expenditure of the fashion industry on creating and managing tech packs is challenging due to the lack of publicly available aggregated data. However, we can estimate based on available information about the costs and time associated with tech pack development. Estimated Costs per Tech Pack The cost of developing a tech pack varies depending on the complexity of the design and the expertise of the designer:\\nShanghai Garment+2theoddfactory.com+2Paromita Das+2 ● Basic Designs\\n: Approximately $100–$500 per tech pack. Shanghai Garment+1makemine.com+1  \\n● Moderately Complex Designs\\n: Around $300–$1,200 per te

In [2]:
a = p8.Agent(Resources)

a('')

2025-06-19 13:38:20.953 | DEBUG    | percolate.services.llm.LanguageModel:call_api_simple:320 - invoking model gpt-4.1, is_streaming=False
2025-06-19 13:38:21.987 | DEBUG    | percolate.services.llm.LanguageModel:__call__:198 - response=<Response [200]>, context=CallingContext(session_id=None, chat_id=None, session_context=None, prefer_json=False, response_model=None, username=None, user_id=None, channel_context=None, channel_ts=None, prefers_streaming=False, is_hybrid_streaming=False, temperature=0.0, plan=None, max_iterations=5, model='gpt-4.1', file_uris=[], streaming_callback=None, response_callback=None)


'Hi! How can I assist you today?'

In [3]:
from percolate.models import Resources, User
p8.repository(User).register()

2025-06-10 21:33:33.640 | WARNING  | percolate.services.PostgresService:execute:595 - Failing to execute query model <class 'percolate.models.p8.types.User'> - verbose_errors=False - relation "User" already exists

2025-06-10 21:33:33.688 | WARNING  | percolate.services.PostgresService:register:464 - The table already exists - will check for schema migration or ignore
2025-06-10 21:33:34.683 | DEBUG    | percolate.services.PostgresService:register:478 - Created embedding table - p8_embeddings."p8_User_embeddings"
2025-06-10 21:33:34.683 | DEBUG    | percolate.services.PostgresService:register:489 - Updating model fields
2025-06-10 21:33:35.220 | DEBUG    | percolate.services.PostgresService:register:494 - Adding the model agent
2025-06-10 21:33:36.320 | INFO     | percolate.services.PostgresService:register:505 - Entity registered


In [ ]:
from percolate.services import PostgresService
pg = PostgresService()
pg.execute("""

-- Function to set user context for row-level security
DROP FUNCTION IF EXISTS p8.set_user_context;
CREATE OR REPLACE FUNCTION p8.set_user_context(
    p_user_id UUID, 
    p_role_level INTEGER = NULL
)
RETURNS VOID AS $$
DECLARE
    v_role_level INTEGER;
    v_user_record RECORD;
    v_groups TEXT[];
BEGIN
    -- If role_level not provided, try to load it from the User table
    IF p_role_level IS NULL THEN
        -- Get role_level, required_access_level, and groups from the User table
        SELECT u.role_level, u.required_access_level, u.groups
        INTO v_user_record
        FROM p8."User" u
        WHERE u.id = p_user_id;
        
        -- Use role_level if available, otherwise use required_access_level
        -- If neither is found, default to public access (100)
        IF v_user_record.role_level IS NOT NULL THEN
            v_role_level := v_user_record.role_level;
        ELSIF v_user_record.required_access_level IS NOT NULL THEN
            -- Use the required_access_level as a fallback
            -- This makes sense because God users have required_access_level=0
            -- Admin users have required_access_level=1, etc.
            v_role_level := v_user_record.required_access_level;
        ELSE
            -- Default to public access if nothing is found
            v_role_level := 100;
        END IF;
        
        -- Get user groups if available
        v_groups := v_user_record.groups;
    ELSE
        -- Use the explicitly provided role level
        v_role_level := p_role_level;
        
        -- Still need to get groups from User table
        SELECT u.groups
        INTO v_groups
        FROM p8."User" u
        WHERE u.id = p_user_id;
    END IF;
    
    -- Set the session variables
    PERFORM set_config('percolate.user_id', p_user_id::TEXT, false);
    PERFORM set_config('percolate.role_level', v_role_level::TEXT, false);
    
    -- Set user groups if available
    IF v_groups IS NOT NULL AND array_length(v_groups, 1) > 0 THEN
        -- For LIKE pattern matching in the policy, we need commas as separators
        PERFORM set_config('percolate.user_groups', ',' || array_to_string(v_groups, ',') || ',', false);
    ELSE
        -- Set empty string if no groups
        PERFORM set_config('percolate.user_groups', '', false);
    END IF;
    
    -- Return the role level as a message for debugging
    RAISE NOTICE 'Set user context: user_id=%, role_level=%, groups=%', 
                 p_user_id, v_role_level, v_groups;
END;
$$ LANGUAGE plpgsql SECURITY DEFINER;


""")

In [ ]:
from percolate.api.controllers.tus import process_pending_s3_resources

await process_pending_s3_resources()

In [ ]:
from percolate.models import   Resources

class ExecutiveResources(Resources):
    class Config:
            namespace:str='executive'
            access_level = 1
            description: str = """Executive docs are used as the Resonance data room and may contain confidential information.
    Certain users that have the system admin or executive roles can access these documents.
    We provide information e.g. for due diligence (investors) or to understand Resonance's 
    - Value proposition
    - Mission
    - Values
    - Strategy
    - IP
    etc.
    
    If you dont know how to answer the question from search you can use other resources
    - Entities can be looked up by keys
    - you can recruit the help of other resources by calling the help function and executing the plan
    """
                
import percolate as p8
p8.repository(ExecutiveResources).register()


In [ ]:
import asyncio
import httpx
import os
import json
#print(os.environ.get('P8_TEST_BEARER_TOKEN'))

async def stream_openai_endpoint(url: str):
    headers = {
        "Authorization": f"Bearer {os.environ.get('P8_TEST_BEARER_TOKEN')}",
        "Accept": "text/event-stream",
        "X-User-Email": "sirsh@resonance.nyc",
        "Content-Type": "application/json"
    }

    payload = {
        "model": 'gpt-4.1-mini',
        "messages": [
            {"role": "user", "content": "By seaching for info on Resonance business model using a function, what can you tell me about please be detailed"}
          #  {"role": "user", "content": "what country was i interested in and what did you tell me?"}
           # {"role": "user", "content": "tell me a long poem 10 verses?"}
        ],
        "stream": True
    }
        
    async with httpx.AsyncClient(timeout=None) as client:
        async with client.stream("POST", url, headers=headers, json=payload) as response:
            async for line in response.aiter_lines():
                #print(line)
                if line.startswith("data: "):
                    data = line[len("data: "):]
                    if data.strip() == "[DONE]":
                        print("[DONE]")
                        break
                    try:
                        a = json.loads(data)['choices'][0]['delta']['content']
                        print(a, end='')
                    except:
                        pass
                        print(data)

# Example usage:
import uuid

session_aka_thread_id = str(uuid.uuid1())
#await stream_openai_endpoint(f"http://localhost:5009/v1/agents/executive-ExecutiveDocs/chat/completions?&session_id={session_aka_thread_id}")
await stream_openai_endpoint(f"http://localhost:5009/v1/agents/executive-ExecutiveDocs/chat/completions")
#await stream_openai_endpoint(f"http://p8.resmagic.io/v1/agents/executive-ExecutiveDocs/chat/completions?&session_id={session_aka_thread_id}")


In [ ]:
from percolate.services.llm.CallingContext import get_user_memory

#get_user_memory('4114f279-f345-511b-b375-1953089e078f')

In [ ]:
from percolate.models import AbstractModel, User
import percolate as p8
#p8.repository(User).register()

In [ ]:
from percolate.interface import try_load_model


In [ ]:
M = p8.try_load_model('executive.ExecutiveDocs')

In [ ]:
from IPython.display import Markdown
#Markdown(p8.Agent(M)("If i wanted to scale REsonance in 2026, what do you think, based on the model, would be a good strategy for breaking into new markets"))
#Markdown(p8.Agent(M)("tell me about the network effects that REsonance can levergage to scale different parts of the business"))
Markdown(p8.Agent(M)("If i were a brand, how would i launch with the REsonance platform. Please provide easy to follow steps", audit_on_flush=True))

In [ ]:

p8.repository(M).search("Saoirse Amarteifio")

In [ ]:
import sys
from warnings import filterwarnings
filterwarnings('ignore')
from percolate.models.p8.types import Resources

sys.path.append('/Users/sirsh/code/res/one-core')

In [ ]:
resources = Resources.chunked_resource(
    uri="/Users/sirsh/Downloads/INST_018.wav",
    parsing_mode="extended",
    chunk_size=1000,
    chunk_overlap=200,
    save_to_db=False,
    
)

len(resources)

In [ ]:

# Simple chunking - basic text extraction (fast, no API costs)
resources = Resources.chunked_resource(
    uri="/Users/sirsh/Downloads/6-Month Roadmap & Milestones.pdf",
    parsing_mode="simple",
    chunk_size=1000,
    chunk_overlap=200,
    save_to_db=False
)

len(resources)

In [ ]:
resources = Resources.chunked_resource(
    uri="/Users/sirsh/Downloads/Transit Insurance Form.pdf", 
    parsing_mode="extended",
    save_to_db=True
)
resources

In [ ]:
resources = Resources.chunked_resource(
    uri="/path/to/document.pdf", 
    parsing_mode="extended",
    chunk_size=1000,
    chunk_overlap=200
)

In [ ]:
"""integration test"""

In [ ]:
from one.services.ask_one import AgentCallingContext
from one.services.slack.models import SlackMessage
from percolate.utils import make_uuid
def _get_userid_from_username(name:str):
    """map the user id from the email address or slack name"""
    from percolate.services import PostgresService
    pg = PostgresService()
    Q = f"""
            SELECT * FROM p8."User" where LOWER(email) = LOWER(%s) or LOWER(slack_id) = LOWER(%s) limit 1
    """
    
    data = pg.execute(Q, data=(name, name))
    if data:
        return data[0]['id']
    
    
channel_ts = f"123445"
message = SlackMessage(user='U01JDKSB196', 
                       text='hi, can you tell me thre things about resonance', 
                       thread_ts="123454", 
                       ts=channel_ts, 
                       channel='C08TE6NNY7L')
"""TODO think about this session id interface"""
ctx = AgentCallingContext(
    channel_context=message.channel,
    streaming_callback=None,#,buffer.add_data,
    username=message.user,

    session_id=make_uuid( message.thread_ts or message.ts), #the slack message thread is used as a session id - when there is no message thread_ts its because we are the originator
    channel_ts=str(channel_ts),
    session_context=f"{message.channel}-{message.thread_ts or message.ts}",
    s3_files=None,
    process_files_async=True,
    user_id=_get_userid_from_username(message.user)
    
)
setattr(ctx,'max_iterations',5)

In [ ]:
import percolate as p8
from percolate.models import PercolateAgent

p8.Agent(PercolateAgent).run('what is a fashion brand',context=ctx)

In [ ]:
#!pip install -U boto3

In [ ]:
from percolate.services import S3Service

s =S3Service()
s

In [ ]:
from io import BytesIO
with open("/Users/sirsh/Downloads/6_Proposed Endeavor Guide and Clarifications.pdf", 'rb') as f:
    data = BytesIO(f.read())
    data.seek(0)
    s.upload_filebytes_to_uri("s3://percolate/test/file.pdf", data)

In [ ]:
s.upload_file_to_uri("s3://percolate/test/file.pdf",
                     "/Users/sirsh/Downloads/6_Proposed Endeavor Guide and Clarifications.pdf" )

In [ ]:
import percolate as p8
from percolate.models.media.tus import register_tus_models
from percolate.models import User
#p8.repository(User).register()
#register_tus_models()
from percolate.services import PostgresService

pg = PostgresService()
 

In [ ]:
pg.execute(""" select * from p8."Resources" order by updated_at desc limit 5 """)

In [ ]:
import requests

session = requests.Session()
session.cookies.set('session', 'eyJzeW5jX2ZpbGVzIjogZmFsc2UsICJ0b2tlbiI6IHsiYWNjZXNzX3Rva2VuIjogInlhMjkuYTBBVzRYdHhocFU5Q1ZwU284R2FwR0JQaHZYcHA0S1BJdGREbW9ocmc3ZFNBbllXdEN4ajM3N2pRcTA1Sk03UnhsR3phSk05cWNkU09SWk5rdjlxckdNV1NfUkVqRFhGMVU1Z3FtN0ZDbEhUb3RUM1N3SldyUmJCZGd3d2N2SWlKbWZmSzZDSWZDemdYNHdJZTQ0ZDFlVFV2emkzc1Y4Z2FTTkstX2tENDVhQ2dZS0FXUVNBUmNTRlFIR1gyTWlHLXRJNXlmYzBFTTA5MnFTMnN6aFpnMDE3NSIsICJleHBpcmVzX2luIjogMzU5OSwgInJlZnJlc2hfdG9rZW4iOiAiMS8vMDVZWDNsZG04WXVJeUNnWUlBUkFBR0FVU053Ri1MOUlyYW1wNHlKR2Y5TnA0UW9aUlVGYkdQeVJXaTlWRHZUX0VLdVJ0cFJ3YnJUcHYxS1FVY1VtOFlBV1hQWURBNnhkckc0TSIsICJzY29wZSI6ICJodHRwczovL3d3dy5nb29nbGVhcGlzLmNvbS9hdXRoL2RyaXZlLnJlYWRvbmx5IGh0dHBzOi8vd3d3Lmdvb2dsZWFwaXMuY29tL2F1dGgvdXNlcmluZm8ucHJvZmlsZSBodHRwczovL3d3dy5nb29nbGVhcGlzLmNvbS9hdXRoL3VzZXJpbmZvLmVtYWlsIGh0dHBzOi8vd3d3Lmdvb2dsZWFwaXMuY29tL2F1dGgvZG9jdW1lbnRzLnJlYWRvbmx5IGh0dHBzOi8vd3d3Lmdvb2dsZWFwaXMuY29tL2F1dGgvZ21haWwucmVhZG9ubHkgb3BlbmlkIiwgInRva2VuX3R5cGUiOiAiQmVhcmVyIiwgImlkX3Rva2VuIjogImV5SmhiR2NpT2lKU1V6STFOaUlzSW10cFpDSTZJalkyTUdWbU0ySTVOemcwWW1SbU5UWmxZbVU0TlRsbU5UYzNaamRtWWpKbE9HTXhZMlZtWm1JaUxDSjBlWEFpT2lKS1YxUWlmUS5leUpwYzNNaU9pSm9kSFJ3Y3pvdkwyRmpZMjkxYm5SekxtZHZiMmRzWlM1amIyMGlMQ0poZW5BaU9pSXhNRGs1TWpNM09EazBPVE0xTFhBemJqZG5hMncyY3prM2FHb3lNRGh1YjNGdE5ISm9PSEpyTTJkaVluWjBMbUZ3Y0hNdVoyOXZaMnhsZFhObGNtTnZiblJsYm5RdVkyOXRJaXdpWVhWa0lqb2lNVEE1T1RJek56ZzVORGt6TlMxd00yNDNaMnRzTm5NNU4yaHFNakE0Ym05eGJUUnlhRGh5YXpOblltSjJkQzVoY0hCekxtZHZiMmRzWlhWelpYSmpiMjUwWlc1MExtTnZiU0lzSW5OMVlpSTZJakV3TWpFMk5UUTNOakV4TkRnd05EQTVOakl6TUNJc0ltVnRZV2xzSWpvaVlXMWhjblJsZVVCbmJXRnBiQzVqYjIwaUxDSmxiV0ZwYkY5MlpYSnBabWxsWkNJNmRISjFaU3dpWVhSZmFHRnphQ0k2SW5OclYyWktjbTVsTlRaMmEyaFRMVnBKVEhOSlNXY2lMQ0p1YjI1alpTSTZJbkp2WlRNM1UzWkJjbmMyU0VwSVVsQTFTbTl0SWl3aWJtRnRaU0k2SWxOaGIybHljMlVnUVcxaGNuUmxhV1pwYnlJc0luQnBZM1IxY21VaU9pSm9kSFJ3Y3pvdkwyeG9NeTVuYjI5bmJHVjFjMlZ5WTI5dWRHVnVkQzVqYjIwdllTOUJRMmM0YjJOTWFWbFFNM0J0TFhSNmFITldURkp1T1ZoaVoxQlFWa2g0U0RCYWVUQmpSMmROZERKcVlVVkJZMnR3VDBsdlRGUkVaejF6T1RZdFl5SXNJbWRwZG1WdVgyNWhiV1VpT2lKVFlXOXBjbk5sSWl3aVptRnRhV3g1WDI1aGJXVWlPaUpCYldGeWRHVnBabWx2SWl3aWFXRjBJam94TnpRM05EQTJPVGszTENKbGVIQWlPakUzTkRjME1UQTFPVGQ5LmM5NWQ5WGtWREFQcjdjU3hBUWVtZzFYWXROSjREcHJhS2JIYlUwb0t1VmNlcDdybTNxQnRqOTZ4c0VnSWc5SXBWVlQ5Ym4zTDF2alU5OTYyVDdQeC1BWEdNYzRab25YX3FYTGc2M1pueGhBVHNIcXdoQ2VoS2N5NVZxYkxkbVVtYnlsZmFYR0pIbFp5ZDhRbkU1RVppY3B1d3ZhN1U5Y3F2Q184ZTVERkN6Ymk5d0V2bHNmc21Vb3U3U0dWdERkX2dqVUxfNTY4RldhSktrNmt3aUhBZXRUaEQ5WmZUaGgybE1vZHdrVHhuOXVzTWhNZ2oyeENvLXR4bjdqa3hRNnk1NGpINE1DRnhUZzk4V2lRdFJvWkxUbm9neXVYbGZzVkI3V3FQdllqTjlrR2tmMnU3SzJCOHFuMkJGT1NjTFJPaHRGSzJDcTIzVm1WZzhNMmVXQlB6dyIsICJyZWZyZXNoX3Rva2VuX2V4cGlyZXNfaW4iOiA2MDQ3OTksICJleHBpcmVzX2F0IjogMTc0NzQxMDU5NiwgInVzZXJpbmZvIjogeyJpc3MiOiAiaHR0cHM6Ly9hY2NvdW50cy5nb29nbGUuY29tIiwgImF6cCI6ICIxMDk5MjM3ODk0OTM1LXAzbjdna2w2czk3aGoyMDhub3FtNHJoOHJrM2diYnZ0LmFwcHMuZ29vZ2xldXNlcmNvbnRlbnQuY29tIiwgImF1ZCI6ICIxMDk5MjM3ODk0OTM1LXAzbjdna2w2czk3aGoyMDhub3FtNHJoOHJrM2diYnZ0LmFwcHMuZ29vZ2xldXNlcmNvbnRlbnQuY29tIiwgInN1YiI6ICIxMDIxNjU0NzYxMTQ4MDQwOTYyMzAiLCAiZW1haWwiOiAiYW1hcnRleUBnbWFpbC5jb20iLCAiZW1haWxfdmVyaWZpZWQiOiB0cnVlLCAiYXRfaGFzaCI6ICJza1dmSnJuZTU2dmtoUy1aSUxzSUlnIiwgIm5vbmNlIjogInJvZTM3U3ZBcnc2SEpIUlA1Sm9tIiwgIm5hbWUiOiAiU2FvaXJzZSBBbWFydGVpZmlvIiwgInBpY3R1cmUiOiAiaHR0cHM6Ly9saDMuZ29vZ2xldXNlcmNvbnRlbnQuY29tL2EvQUNnOG9jTGlZUDNwbS10emhzVkxSbjlYYmdQUFZIeEgwWnkwY0dnTXQyamFFQWNrcE9Jb0xURGc9czk2LWMiLCAiZ2l2ZW5fbmFtZSI6ICJTYW9pcnNlIiwgImZhbWlseV9uYW1lIjogIkFtYXJ0ZWlmaW8iLCAiaWF0IjogMTc0NzQwNjk5NywgImV4cCI6IDE3NDc0MTA1OTd9fSwgInNlc3Npb25faWQiOiAiYzU4ZmQ1MWYtNDAzZC00MjU3LTllMTgtMTc3MzdlNDRmZDNmIn0=.aCdQlQ.qHKMWsxfHuBLHnk3Ck3Ux_tiqp8')

response = session.get('http://localhost:5008/auth/ping')
print(response.json())

response = session.get('http://localhost:5008/auth/session/info')
print(response.json())

In [ ]:
session.get('http://localhost:5008/tus/user/uploads?query=anything').json()

In [ ]:
response = session.get('http://localhost:5008/auth/me')
print(response.json())

In [ ]:
from percolate.api.routes.auth.utils import get_user_from_email, is_valid_token_for_user, extract_user_info_from_token
extract_user_info_from_token('eyJhbGciOiJSUzI1NiIsImtpZCI6IjY2MGVmM2I5Nzg0YmRmNTZlYmU4NTlmNTc3ZjdmYjJlOGMxY2VmZmIiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJhenAiOiIxMDk5MjM3ODk0OTM1LXAzbjdna2w2czk3aGoyMDhub3FtNHJoOHJrM2diYnZ0LmFwcHMuZ29vZ2xldXNlcmNvbnRlbnQuY29tIiwiYXVkIjoiMTA5OTIzNzg5NDkzNS1wM243Z2tsNnM5N2hqMjA4bm9xbTRyaDhyazNnYmJ2dC5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjEwMjE2NTQ3NjExNDgwNDA5NjIzMCIsImVtYWlsIjoiYW1hcnRleUBnbWFpbC5jb20iLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiYXRfaGFzaCI6ImlBS2ZoVXRMU1ZDbkR4aVE1X3NwUUEiLCJub25jZSI6IjhGamhCTE5TYzBFeDJ2Qm5KSVA1IiwibmFtZSI6IlNhb2lyc2UgQW1hcnRlaWZpbyIsInBpY3R1cmUiOiJodHRwczovL2xoMy5nb29nbGV1c2VyY29udGVudC5jb20vYS9BQ2c4b2NMaVlQM3BtLXR6aHNWTFJuOVhiZ1BQVkh4SDBaeTBjR2dNdDJqYUVBY2twT0lvTFREZz1zOTYtYyIsImdpdmVuX25hbWUiOiJTYW9pcnNlIiwiZmFtaWx5X25hbWUiOiJBbWFydGVpZmlvIiwiaWF0IjoxNzQ3NTk4Mjk0LCJleHAiOjE3NDc2MDE4OTR9.P0goYoml9dxdPHiH-f7IjeEFWMv6qDT6cv-GxHma2drk14pkIuwwdU0w2OCDSlFRGc4hfePIvMU3KIddNPGnkc8qvBKDNvafdBkndMQ0D1rae8BNScVr3EuZiikv_O54Eff1Lv6cmcKd978m5I7p8xVx-2aGGrxGELz3OFszgnh6POo1efiuznp6sPIQJbskfI1a8YR1_GArU9jkwhjhczEsibWNtLn5hIzox02qBTdjfrh69eOOKSmysb2SjWl54AmWxubYfR1M1IO0CAPHUQFzVrdouUsBvZnIk3xWnF-roSMrt68aoWWCtzAxtupzy3ziY8RdTVyWy7uEnr-k6g')

In [ ]:
import requests

session = requests.Session()
#session.cookies.set('session', 'eyJhbGciOiJSUzI1NiIsImtpZCI6IjY2MGVmM2I5Nzg0YmRmNTZlYmU4NTlmNTc3ZjdmYjJlOGMxY2VmZmIiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJhenAiOiIxMDk5MjM3ODk0OTM1LXAzbjdna2w2czk3aGoyMDhub3FtNHJoOHJrM2diYnZ0LmFwcHMuZ29vZ2xldXNlcmNvbnRlbnQuY29tIiwiYXVkIjoiMTA5OTIzNzg5NDkzNS1wM243Z2tsNnM5N2hqMjA4bm9xbTRyaDhyazNnYmJ2dC5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjEwMjE2NTQ3NjExNDgwNDA5NjIzMCIsImVtYWlsIjoiYW1hcnRleUBnbWFpbC5jb20iLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiYXRfaGFzaCI6InNlN2FuOVhHd19BeEVGdkFOSTZSckEiLCJub25jZSI6InZWRlVBekpTU3RsMGZmQnp3RXBsIiwibmFtZSI6IlNhb2lyc2UgQW1hcnRlaWZpbyIsInBpY3R1cmUiOiJodHRwczovL2xoMy5nb29nbGV1c2VyY29udGVudC5jb20vYS9BQ2c4b2NMaVlQM3BtLXR6aHNWTFJuOVhiZ1BQVkh4SDBaeTBjR2dNdDJqYUVBY2twT0lvTFREZz1zOTYtYyIsImdpdmVuX25hbWUiOiJTYW9pcnNlIiwiZmFtaWx5X25hbWUiOiJBbWFydGVpZmlvIiwiaWF0IjoxNzQ3NTkzNjk4LCJleHAiOjE3NDc1OTcyOTh9.Z61fPlHZ__DZ38rhlU-PmloMjfs6Tj2xLFqgLj5r2yOHsPDUSEmBYm8Ay7qEmUj5kWV-iyHRgW1c7Z-p3z1k8KUyPVsKXXbZlHvlHUDBmTQo8Uy3axHyei02Vz2ejImBspjlLzjg1bf_wQUXOo5pAdR8D0elaWoRLseslRvyLdZf3k2mk_pxy3c3utkefco1psIaGcdRC44OIdqCCWtPh1MXsSu0fMW4vsdpJQYRrIlmodMWwTLph6Rv2jJ6Rl5JQU3OG6Bx9dW5XQld1k1eQuP2_UEij4-lo3_JsJD4zdbFDSAbiNUbZICof6tatbsDiCpHd3o9-AV_BBlcn_lKdA')

session.cookies.set('session', 'eyJhbGciOiJSUzI1NiIsImtpZCI6IjY2MGVmM2I5Nzg0YmRmNTZlYmU4NTlmNTc3ZjdmYjJlOGMxY2VmZmIiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJhenAiOiIxMDk5MjM3ODk0OTM1LXAzbjdna2w2czk3aGoyMDhub3FtNHJoOHJrM2diYnZ0LmFwcHMuZ29vZ2xldXNlcmNvbnRlbnQuY29tIiwiYXVkIjoiMTA5OTIzNzg5NDkzNS1wM243Z2tsNnM5N2hqMjA4bm9xbTRyaDhyazNnYmJ2dC5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjEwMjE2NTQ3NjExNDgwNDA5NjIzMCIsImVtYWlsIjoiYW1hcnRleUBnbWFpbC5jb20iLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiYXRfaGFzaCI6Im5KcnJIMm9lanZvSHRrSUJyS1FLckEiLCJub25jZSI6Imc1bVk5Z3AyaVduSDF6Q2U2MFl5IiwibmFtZSI6IlNhb2lyc2UgQW1hcnRlaWZpbyIsInBpY3R1cmUiOiJodHRwczovL2xoMy5nb29nbGV1c2VyY29udGVudC5jb20vYS9BQ2c4b2NMaVlQM3BtLXR6aHNWTFJuOVhiZ1BQVkh4SDBaeTBjR2dNdDJqYUVBY2twT0lvTFREZz1zOTYtYyIsImdpdmVuX25hbWUiOiJTYW9pcnNlIiwiZmFtaWx5X25hbWUiOiJBbWFydGVpZmlvIiwiaWF0IjoxNzQ3NTk4NTcwLCJleHAiOjE3NDc2MDIxNzB9.NpUvgWfPSpQLaQL8UR1mr2Vd4S7-3ZLEoeK2Dbpbi_uBES51dm8MUFnDTQPNULwo-IVUE9GBAr5HInqTx6L95cTiAohTUG-Qzio96rwtnNYF0YWkBByhTKWRKhdQ6YK97TwGffZMrEzaQ_QvHe_XF17exJpKkRLtjijCuMmmAMMqbdgtsd5XVtxSma_HFapIi4dTcZOiVqWglzsGCXWxvZqtWXiHii1niKFYb95aF8q4sRgsRGs4Z7cCc1l6YzD8sSnkTPyay_qWuAWvMlS3iY9x3777zG82U9Fi38sZ2IaQ7CS1IbZ7CrBo45-2iuh28PSw7H0v2qGcTwfOF0lhTA')

response = session.get('https://eepis.percolationlabs.ai/auth/ping')
print(response.json())

response = session.get('https://eepis.percolationlabs.ai/auth/session/info')
print(response.json())

In [ ]:
session.get('https://eepis.percolationlabs.ai/tus/user/uploads?limit=10').json()

In [ ]:
session.get('https://eepis.percolationlabs.ai/tus/user/uploads?query=CevolvingAnyibodies.').json()

In [ ]:
session.get('https://eepis.percolationlabs.ai/tus/user/uploads?query=Eta Carinae pulsed in the distance.').json()